Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad средствами библиотеки Pandas сформируйте поле выводов по каждому анализирумомому параметру. Например, по показателю pH получен результат 8.4 единицы pH при нормативе от 6 до 9 единиц pH. Т.о. по данному показателю результат анализа в норме. Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, 6 >= pH >= 9. В итоговом DataFrame столбец "Показатель" сделайте индексным.
Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).

In [4]:
import pandas as pd
import numpy as np
import requests
import io
import re

Для начала сделаем столбец "Показатель" индексным и посмотрим дата фрейм

In [190]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')), index_col=0) #делаем столбец "Показатель" индексным
display(df)

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,б/цвета,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


Уберем пару лишних столбцов

In [191]:
df.pop('Единица измерений') #уберем из дата фрейма столбец "Единица измерений", так как он не несёт для нас полезной информации
df.pop('Норматив')#Норматив убираем тоже, вместо него создадим новый список с референсными значениями показателей

Показатель
pH                                        в пределах 6-9
Запах                                       не более 2-3
Цветность                                    не более 30
Жёсткость                                в пределах 7-10
Аммиак и аммоний-ион (по азоту)             не более 1,5
Нитриты (по NO2)                            не более 3,3
Нитраты (по NO3)                             не более 45
Фосфаты (P)                                 не более 3,5
Хлориды (Cl)                                не более 350
Сульфаты (SO4)                              не более 500
Железо (включая хлорное железо) по Fe       не более 0,3
Нефть                                       не более 0,3
Общая минерализация (сухой остаток)        не более 1000
Окисляемость перманганатная                 не более 0,5
Name: Норматив, dtype: object

Создадим два списка с референсными значениями для анализа показателей воды

In [192]:
list_norma_min = [6, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
list_norma_max = [9, 3, 30, 10, 1.5, 3.3, 45, 3.5, 350, 500, 0.3, 0.3, 1000, 0.5] #создаем список референсных значений для параметров воды

Добавляем в наш дата фрейм два получившихся столбца с минимальными и максимальными значениями показателей

In [193]:
df['Min']=list_norma_min
df['Max']=list_norma_max

Создаем списки в качестве элементов которых выступают значения дата фрейма. Тип данных меняем со строки на число с плавающей запятой для дальнейшего сравнения, так как нельзя сравнить строчные значения. Список А - это анализируемые значение, Список В - минимальные референсные значения и Список С - максимальные референсные значения.

In [194]:
list_a=[]
list_a.append(float(df.iloc[0,0]))
list_a.append(float(df.iloc[1,0]))
list_a.append(0) #исправляем ошибку в данных, присваиваем цифровое значение
list_a.append(float(df.iloc[3,0]))
list_a.append(float(df.iloc[4,0]))
list_a.append(float(df.iloc[5,0]))
list_a.append(float(df.iloc[6,0]))
list_a.append(float(df.iloc[7,0]))
list_a.append(float(df.iloc[8,0]))
list_a.append(float(df.iloc[9,0]))
list_a.append(float(df.iloc[10,0]))
list_a.append(float(df.iloc[11,0]))
list_a.append(float(df.iloc[12,0]))
list_a.append(float(df.iloc[13,0]))

In [195]:
list_b = []
list_b.append(float(df.iloc[0,1]))
list_b.append(float(df.iloc[1,1]))
list_b.append(float(df.iloc[2,1]))
list_b.append(float(df.iloc[3,1]))
list_b.append(float(df.iloc[4,1]))
list_b.append(float(df.iloc[5,1]))
list_b.append(float(df.iloc[6,1]))
list_b.append(float(df.iloc[7,1]))
list_b.append(float(df.iloc[8,1]))
list_b.append(float(df.iloc[9,1]))
list_b.append(float(df.iloc[10,1]))
list_b.append(float(df.iloc[11,1]))
list_b.append(float(df.iloc[12,1]))
list_b.append(float(df.iloc[13,1]))

In [196]:
list_c=[]
list_c.append(float(df.iloc[0,2]))
list_c.append(float(df.iloc[1,2]))
list_c.append(float(df.iloc[2,2]))
list_c.append(float(df.iloc[3,2]))
list_c.append(float(df.iloc[4,2]))
list_c.append(float(df.iloc[5,2]))
list_c.append(float(df.iloc[6,2]))
list_c.append(float(df.iloc[7,2]))
list_c.append(float(df.iloc[8,2]))
list_c.append(float(df.iloc[9,2]))
list_c.append(float(df.iloc[10,2]))
list_c.append(float(df.iloc[11,2]))
list_c.append(float(df.iloc[12,2]))
list_c.append(float(df.iloc[13,2]))


Создаем пустой список и сравниваем значения

In [211]:
water_check_list = []

In [212]:
for i in range(0,14):
    if list_a[i]>=list_b[i] and list_a[i]<=list_c[i]:
        water_check_list.append("Да")
    else:
        water_check_list.append("Нет")

In [213]:
print(water_check_list)

['Да', 'Да', 'Да', 'Да', 'Да', 'Да', 'Да', 'Да', 'Да', 'Да', 'Да', 'Нет', 'Да', 'Нет']


Добавляем полученный список в качестве столбца в наш дата фрейм

In [214]:
df['Соответствие']= water_check_list

Убираем лишние столбцы из нашего дата фрейма

In [215]:
df.pop('Результат анализа') #уберем из дата фрейма столбец "Единица измерений", так как он не несёт для нас полезной информации
df.pop('Min')
df.pop('Max')
df

,Соответствие
Показатель,
pH,Да
Запах,Да
Цветность,Да
Жёсткость,Да
Аммиак и аммоний-ион (по азоту),Да
Нитриты (по NO2),Да
Нитраты (по NO3),Да
Фосфаты (P),Да
Хлориды (Cl),Да


Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

In [258]:
box_fruit = ["яблоко", "яблоко","яблоко","яблоко","апельсин", "апельсин", "апельсин","апельсин","апельсин"]
N = 3
cases = []
i=1
while i <= N:
    fruit = np.random.choice(box_fruit)
    cases.append(fruit)
    i+=1
        
display("Яблоко достали: {} раз".format(len(pd.DataFrame(cases, columns=["REZ"]).query('REZ =="яблоко"'))))
display("Апельсин достали: {} раз".format(len(pd.DataFrame(cases, columns=["REZ"]).query('REZ =="апельсин"'))))

'Яблоко достали: 0 раз'

'Апельсин достали: 3 раз'

Аналитически можно посчитать как произведение вероятностей 5/9 на 4/8 на 3/7 = 0,119. На практике вероятность выпадения кажется чаще.
Также можно сделать другим способом проще:

In [267]:
box_fruit2 = np.array(["яблоко","апельсин"])
probabilities = [0.45, 0.55]
np.random.choice(box_fruit2, 3, p=probabilities)

array(['апельсин', 'апельсин', 'апельсин'], dtype='<U8')

Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?

В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [78]:
detail_box = ["Стандартная", "Стандартная","Стандартная","Стандартная","Стандартная","Стандартная","Стандартная","Нестандартная","Нестандартная","Нестандартная", ]
cases1 = []
for number in range(10):
    number = number+1
    detail = np.random.choice(detail_box)
    cases1.append(detail)
    if detail == "Нестандартная":
        pass
    elif detail == "Стандартная":
        break
    
   
print(cases1)

['Нестандартная', 'Стандартная']
